<a href="https://colab.research.google.com/github/minjaekim070/Bootcamp_Project/blob/main/CNN_project_1%EC%9D%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

전체 데이터셋의 다운로드가 오래 걸려서 bottle 폴더에 대한 내용을 우선적으로 정상/비정상으로 분류할 수 있도록 진행

#Library import


In [41]:
# 패키지 import
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf
from torchvision import transforms, datasets, utils
from torchsummary import summary

import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras.preprocessing import image


#이미지 업로드 및 배열 변환


In [65]:
# 이미지 경로와 카테고리 설정
data_dir = '/content/drive/MyDrive/bottle/train/good'

# 이미지 로딩 및 배열 변환
train_image_data = []
for img_name in os.listdir(data_dir):
    img_path = os.path.join(data_dir, img_name)
    img = image.load_img(img_path, target_size=(32, 32))  # 이미지를 지정된 크기로 로드합니다
    img_array = image.img_to_array(img)  # 이미지를 배열 형태로 변환합니다
    train_image_data.append(img_array)


In [66]:
# 배열로 변환
train_image_data = np.array(train_image_data)

In [67]:
train_image_data.shape

(209, 32, 32, 3)

In [45]:
large_data_dir = '/content/drive/MyDrive/bottle/test/broken_large'
small_data_dir = '/content/drive/MyDrive/bottle/test/broken_small'
contamination_data_dir='/content/drive/MyDrive/bottle/test/contamination'
test_data_dir = '/content/drive/MyDrive/bottle/test/good'

# 이미지 로딩 및 전처리를 위한 ImageDataGenerator 생성
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255)  # 이미지를 0과 1 사이의 값으로 정규화합니다

# test 이미지 로딩 및 배열 변환
test_image_data = []
for img_name in os.listdir(test_data_dir):
    img_path = os.path.join(test_data_dir, img_name)
    img = image.load_img(img_path, target_size=(32, 32))  # 이미지를 지정된 크기로 로드합니다
    img_array = image.img_to_array(img)  # 이미지를 배열 형태로 변환합니다
    test_image_data.append(img_array)

large_image_data = []
for img_name in os.listdir(large_data_dir):
    img_path = os.path.join(large_data_dir, img_name)
    img = image.load_img(img_path, target_size=(32, 32))  # 이미지를 지정된 크기로 로드합니다
    img_array = image.img_to_array(img)  # 이미지를 배열 형태로 변환합니다
    large_image_data.append(img_array)

small_image_data = []
for img_name in os.listdir(small_data_dir):
    img_path = os.path.join(small_data_dir, img_name)
    img = image.load_img(img_path, target_size=(32, 32))  # 이미지를 지정된 크기로 로드합니다
    img_array = image.img_to_array(img)  # 이미지를 배열 형태로 변환합니다
    small_image_data.append(img_array)

contamination_image_data = []
for img_name in os.listdir(contamination_data_dir):
    img_path = os.path.join(contamination_data_dir, img_name)
    img = image.load_img(img_path, target_size=(32, 32))  # 이미지를 지정된 크기로 로드합니다
    img_array = image.img_to_array(img)  # 이미지를 배열 형태로 변환합니다
    contamination_image_data.append(img_array)

# 배열로 변환
test_image_data = np.array(test_image_data)
small_image_data = np.array(small_image_data)
large_image_data = np.array(large_image_data)
contamination_image_data = np.array(contamination_image_data)

#모델 구성 및 결과 예측

In [48]:
import tensorflow as tf
from tensorflow.keras import layers

# CNN 모델 생성
model = tf.keras.Sequential()

# 첫 번째 컨볼루션 레이어
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))

# 두 번째 컨볼루션 레이어
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))

# 세 번째 컨볼루션 레이어
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))

# 전체 데이터를 1D 벡터로 변환
model.add(layers.Flatten())

# 완전 연결 레이어
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))

# 출력 레이어
model.add(layers.Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모델 요약 정보 출력
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_19 (Conv2D)          (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 15, 15, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 15, 15, 64)        18496     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 7, 7, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 7, 7, 128)         73856     
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 3, 3, 128)       

In [49]:
# 입력 데이터와 레이블 준비
train_images = image_data
train_labels = np.ones((len(train_images), 1))  # 정상 데이터이므로 모두 1로 레이블을 설정

# 모델 학습
model.fit(train_images, train_labels, epochs=10, batch_size=32)

Epoch 1/10
7/7 [==============================] - 2s 69ms/step - loss: 5.2810 - accuracy: 0.8612
Epoch 2/10
7/7 [==============================] - 0s 64ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/10
7/7 [==============================] - 0s 66ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/10
7/7 [==============================] - 0s 65ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/10
7/7 [==============================] - 0s 65ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/10
7/7 [==============================] - 0s 63ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/10
7/7 [==============================] - 0s 66ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/10
7/7 [==============================] - 0s 71ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/10
7/7 [==============================] - 0s 67ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/10
7/7 [==============================] - 0s 67ms/step - loss: 0.0000e+00 - accuracy: 

In [ ]:
predictions = model.predict(test_image_data)
# 예측 결과 출력
for i, prediction in enumerate(predictions):
    if prediction >= 0.5:
        print(f"이미지 {i+1}: 정상")
    else:
        print(f"이미지 {i+1}: 비정상")

In [ ]:
predictions

In [ ]:
small_predictions = model.predict(small_image_data)
large_predictions = model.predict(large_image_data)
contamination_predictions = model.predict(contamination_image_data)

print(small_predictions)

In [ ]:
# 예측 결과 출력
print('small_large data set에 대한 예측 결과')
for i, prediction in enumerate(small_predictions):
    if prediction >= 0.5:
        print(f"이미지 {i+1}: 정상")
    else:
        print(f"이미지 {i+1}: 비정상")

In [ ]:
# 예측 결과 출력
print('broken_large data set에 대한 예측 결과')
for i, prediction in enumerate(large_predictions):
    if prediction >= 0.5:
        print(f"이미지 {i+1}: 정상")
    else:
        print(f"이미지 {i+1}: 비정상")

In [ ]:
# 예측 결과 출력
print('contamination data set에 대한 예측 결과')
for i, prediction in enumerate(contamination_predictions):
    if prediction >= 0.5:
        print(f"이미지 {i+1}: 정상")
    else:
        print(f"이미지 {i+1}: 비정상")